In [1]:
import pandas as pd
from nltk.corpus import comparative_sentences  #keyword comparison 생성 모듈

data = pd.read_csv('reviews_preprocessing.csv', engine='python')
data.head()

,asin,date,product,review,score,user
0,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i have taken over 500 pics using all three foc...,4.0,A.J.Kirchoff (xgi@ametro.net)
1,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i wish that it would take the 16memory and th...,4.0,A.J.Kirchoff (xgi@ametro.net)
2,B00000J48G,"06 10, 2002",Polaroid PDC-640 0.3MP Digital Camera Creative...,only draw back is theres no optical lens.,4.0,"Andrea Rowland ""Andrea"""
3,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,i like the way my 5x7 prints and smaller turn...,4.0,"""denisey65"""
4,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,"flashing is important in any photography, and...",4.0,"""denisey65"""


In [2]:
#데이터 확인
len(data)

442911

In [33]:
#kw 사전 리스트 생성
### kw_list = comparative_sentences.keywords_readme()[5:]
### kw_str = '|'.join(kw_list)

#kw 사전 수작업으로 분류 후 다시 읽어오기
def call_kw():
    kw_list = pd.read_excel('kw_list.xls')
    #칼럼 중에 'to_use'만 가져오기
    kw_list = list(kw_list['to_use'])
#     kw_str = '|'.join(kw_list)
    
    return kw_list

for kw in call_kw():
    print(kw)

beat
inferior
outstrip
Choice
choose
prefer
recommend
outperform
superior
all
favor
defeat
twice
thrice
Number one
more
like
Versus
first
outdistance
before
double
outsell
nobody
Vs
last
after
thrice
improve
only
outmatch
ahead
fraction
outdo
unmatched
peerless
differ
outwit
Compare
top
exceed
lead
win
outstrip
none
unrivaled
dominate
second
nonpareil
advantage
unlike
least
outclass
outfox
outdistance
most
up against
less
half
behind
Compete


In [50]:
# kw_list=call_kw()
# #for문 기반 cs,ss 라벨링
# def make_sscs(data):
#     from nltk import word_tokenize, pos_tag
#     cs_list = []
#     ss_list = []
#     for i in range(len(data)):
#         text = data.loc[i, 'review']
#         product = data.loc[i, 'product']
        
#         # nltk로 토크나이즈, 포스태깅
#         tokens = word_tokenize(text)
#         pos_tokens = pos_tag(tokens)
        
#         # 비교구문이 들어있는 문장 찾기
#         count = 0
#         for tok, pos in pos_tokens:
#             if pos in ['JJR', 'JJS', 'RBS', "RBR"]:
#                 count +=1

#         if count != 0:
#             cs_list.append(i)
#         elif count == 0:
#             ss_list.append(i)
        
    
#     df_ss = data.loc[ss_list]     
    
#     df_cs = data.loc[cs_list]
#     # product_list가 없어서 #처리함. 추후 보강해야 함
# #     product_list = []
# #     products = '|'.join(product_list)
# #     df = df[df.review.str.contains(products)== True]
#     df_cs = df_cs[df_cs.review.str.contains(kw_str)== True]
    
#     df_cs['CS_SS'] = 'CS'
#     df_ss['CS_SS'] = 'SS'

#     df_t = df_cs.append(df_ss, ignore_index=True)
#     return df_t

In [53]:
kw_list=call_kw()
def tokenizedNtagging(text):
    #kw로 분류
    for kw in kw_list:
        if kw.lower() in text:
            return 'CS'
        
    #pos로 분류    
    from nltk import word_tokenize, pos_tag
    # nltk로 토크나이즈, 포스태깅
    tokens = word_tokenize(text)
    pos_tokens = pos_tag(tokens)

    count = 0
    for tok, pos in pos_tokens:
        if pos in ['JJR', 'JJS', 'RBS', "RBR"]:
            count+=1

    if count != 0:
        return 'CS'
    else:
        return 'SS'            

In [54]:
def make_sscs(data):
    data['CS_SS']=data['review']
    data['CS_SS']=data['CS_SS'].apply(tokenizedNtagging)
    
    return data

In [55]:
df_t = make_sscs(data)
df_t.head()

,asin,date,product,review,score,user,CS_SS
0,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i have taken over 500 pics using all three foc...,4.0,A.J.Kirchoff (xgi@ametro.net),CS
1,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i wish that it would take the 16memory and th...,4.0,A.J.Kirchoff (xgi@ametro.net),SS
2,B00000J48G,"06 10, 2002",Polaroid PDC-640 0.3MP Digital Camera Creative...,only draw back is theres no optical lens.,4.0,"Andrea Rowland ""Andrea""",CS
3,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,i like the way my 5x7 prints and smaller turn...,4.0,"""denisey65""",CS
4,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,"flashing is important in any photography, and...",4.0,"""denisey65""",CS


In [56]:
len(df_t[df_t.CS_SS=='CS'])

263792

In [57]:
len(df_t[df_t.CS_SS=='SS'])

179119

In [58]:
df_t.to_csv('labelingSSCS.csv',index=False, encoding='utf-8')